# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [ ]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

training_file = 'traffic-signs-data/train.p'
testing_file = 'traffic-signs-data/test.p'
validation_file= 'traffic-signs-data/valid.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

X_train, y_train, sizes_train, coords_train = train['features'], train['labels'], train['sizes'], train['coords']    
X_valid, y_valid, sizes_valid, coords_valid = valid['features'], valid['labels'], valid['sizes'], valid['coords']    
X_test,  y_test,  sizes_test,  coords_test  = test['features'], test['labels'], test['sizes'], test['coords']    

import csv
sign_names_file = 'signnames.csv'
with open(sign_names_file) as sign_csv_file:
    sign_csv_file = csv.reader(sign_csv_file, delimiter=',')
    sign_names = []
    for row in sign_csv_file:
        sign_name = row[1]
        if sign_name != 'SignName':
            sign_names.append(row[1])
            

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

import numpy as np

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of testing examples.
n_test = len(X_test)

# Number of validating examples.
n_valid = len(X_valid)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape
image_size = image_shape[0]

# TODO: How many unique classes/labels there are in the datasets.
sign_classes_train, class_indices_train, class_counts_train = np.unique(y_train, return_index = True, return_counts = True)
sign_classes_valid, class_indices_valid, class_counts_valid = np.unique(y_valid, return_index = True, return_counts = True)
sign_classes_test, class_indices_test, class_counts_test = np.unique(y_test, return_index = True, return_counts = True)

n_classes = len(sign_classes_train)

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Number of valid examples =", n_valid)
print("Image data shape =", image_shape)
print("Image data size =", image_size)
print("Number of classes =", n_classes)


### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

import matplotlib.image as mpimg


In [ ]:
import random

n_sign_show = 10
# sign index, name and images
print("SIGN TRAIN IMAGES")
for sign_index in range(n_classes):
    sign_sample_size = class_counts_train[sign_index]
    print("class {0:2} : {1:50} : {2} samples".format(sign_index, sign_names[sign_index], sign_sample_size))
    fig = plt.figure(figsize = (12, 2))
    fig.subplots_adjust(left = 0, right = 1, bottom = 0, top = 1, hspace = 0.1, wspace = 0.1)
    class_index = class_indices_train[sign_index]
    random_indices = random.sample(range(class_index, class_index + sign_sample_size), n_sign_show)
    for index in range(n_sign_show):
        axis = fig.add_subplot(1, 10, index + 1, xticks = [], yticks = [])
        axis.imshow(X_train[random_indices[index]])
    plt.show()
    print('===============================================================================================================')


In [ ]:
print("SIGN OCCURRENCES IN DATASETS")
N = n_classes
ind = np.arange(n_classes)  # the x locations for the groups
width = 0.2       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, class_counts_train, width, color='r')
rects2 = ax.bar(ind + width, class_counts_valid, width, color='g')
rects3 = ax.bar(ind + 2 * width, class_counts_test, width, color='b')

# add some text for labels, title and axes ticks
ax.set_ylabel('Occurrences')
ax.set_title('SIGN OCCURRENCES IN DATASETS')
ax.set_xticks(ind + width)
x_tick_labels = []
for index in range(n_classes):
    if index % 5 == 0:
        x_tick_labels.append(str(index))
    else:
        x_tick_labels.append('')
ax.set_xticklabels(x_tick_labels)

ax.legend((rects1[0], rects2[0], rects3[0]), ('train', 'valid', 'test'))


def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

#autolabel(rects1)
#autolabel(rects2)
#autolabel(rects3)
ax.axis("on")

plt.show()


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. Preprocessing steps could include normalization, converting to grayscale, etc.
### Feel free to use as many code cells as needed.

In [ ]:
# convert to gray scale
# some images are dark; correct the exposure.
from sklearn.utils import shuffle
from skimage import exposure
import skimage
import cv2

def convert_to_grayscale_and_correct_intensity(X, y = None):
    # convert rgb to ycbcr - but only need y = gray scale
    X = 0.299 * X[:, :, :, 0] + 0.587 * X[:, :, :, 1] + 0.114 * X[:, :, :, 2]

    # scale features to be in [0, 1]
    X = (X / 255).astype(np.float32)
    
    # apply localized histogram localization
    for index in range(X.shape[0]):
        X[index] = skimage.exposure.equalize_adapthist(X[index])

    # add single gray scale channel
    X = X.reshape(X.shape + (1,))
    return X, y

In [ ]:
X_train_gray, y_train_gray = convert_to_grayscale_and_correct_intensity(X_train, y_train)
X_valid_gray, y_valid_gray = convert_to_grayscale_and_correct_intensity(X_valid, y_valid)
X_test_gray, y_test_gray = convert_to_grayscale_and_correct_intensity(X_test, y_test)

In [ ]:
print(X_train.shape)
print(X_train_gray.shape)

In [ ]:
n_sign_show = 10
# sign index, name and images
print("SIGN TRAIN IMAGES")
for sign_index in range(n_classes):
    sign_sample_size = class_counts_train[sign_index]
    print("class {0:2} : {1:50} : {2} samples".format(sign_index, sign_names[sign_index], sign_sample_size))
    fig = plt.figure(figsize = (12, 2))
    fig.subplots_adjust(left = 0, right = 1, bottom = 0, top = 1, hspace = 0.1, wspace = 0.1)
    class_index = class_indices_train[sign_index]
    random_indices = random.sample(range(class_index, class_index + sign_sample_size), n_sign_show)
    for index in range(n_sign_show):
        axis = fig.add_subplot(1, 10, index + 1, xticks = [], yticks = [])
        axis.imshow(X_train_gray[random_indices[index]].reshape(image_size, image_size), cmap='gray')
    plt.show()
    print('===============================================================================================================')


In [ ]:
# several sign classes are under represented
# horizontally and vertically reflect images to increase dataset
def reflect(X, y):
    # sign classes when horizontally reflected, are still the same sign classes
    self_horizontally_reflect = np.array([11, 12, 13, 15, 17, 18, 22, 26, 30, 35])
    # sign classes when vertically reflected, are still the same sign classes
    self_vertictally_reflect = np.array([1, 5, 12, 15, 17])
    # sign classes when both horizontally and vertically reflected, are still the same sign classes
    self_both_reflect = np.array([32, 40])
    # sign classes when horizontally reflected become different sign class but are still useful
    different_horizontally_reflect = np.array([
            [19, 20],
            [20, 19],
            [33, 34],
            [34, 33],
            [36, 37],
            [37, 36],
            [38, 39],
            [39, 38]
        ])
    num_classes = n_classes
    
    X_extended = np.empty([0, X.shape[1], X.shape[2], X.shape[3]], dtype = X.dtype)
    y_extended = np.empty([0], dtype = y.dtype)
    
    for c in range(num_classes):
        # copy the original data for this class
        X_extended = np.append(X_extended, X[y == c], axis = 0)
        
        # possible to horizontally reflect image, would result belong to same class?
        if c in self_horizontally_reflect:
            # copy horizontally relfected image to same class extended array
            X_extended = np.append(X_extended, X[y == c][:, :, ::-1, :], axis = 0)

        # possible to horizontally reflect image, would result belong to different class?
        if c in different_horizontally_reflect[:, 0]:
            # copy horizontally relfected image to different class extended array
            reflected_class = different_horizontally_reflect[different_horizontally_reflect[:, 0] == c][0][1]
            X_extended = np.append(X_extended, X[y == reflected_class][:, :, ::-1, :], axis = 0)
        # fill labels for extended images to current class
        y_extended = np.append(y_extended, np.full((X_extended.shape[0] - y_extended.shape[0]), c, dtype = int))
        
        # possible to vertically reflect image, would result belong to same class?
        if c in self_vertictally_reflect:
            # copy vertically relfected image to same class extended array
            X_extended = np.append(X_extended, X_extended[y_extended == c][:, ::-1, :, :], axis = 0)
        # fill labels for extended images to current class
        y_extended = np.append(y_extended, np.full((X_extended.shape[0] - y_extended.shape[0]), c, dtype = int))

        # possible to horizontally and vertically reflect image, would result belong to same class?
        if c in self_both_reflect:
            # copy horizontally and vertically relfected image to same class extended array
            X_extended = np.append(X_extended, X_extended[y_extended == c][:, ::-1, ::-1, :], axis = 0)
        # fill labels for extended images to current class
        y_extended = np.append(y_extended, np.full((X_extended.shape[0] - y_extended.shape[0]), c, dtype = int))

    return X_extended, y_extended


In [ ]:
X_train_reflect, y_train_reflect = reflect(X_train_gray, y_train_gray)
X_valid_reflect, y_valid_reflect = reflect(X_valid_gray, y_valid_gray)
X_test_reflect, y_test_reflect = reflect(X_test_gray, y_test_gray)

In [ ]:
n_train_reflect = len(X_train_reflect)
n_test_reflect = len(X_test_reflect)
n_valid_reflect = len(X_valid_reflect)
print("Number of training examples =", n_train_reflect)
print("Number of testing examples =", n_test_reflect)
print("Number of valid examples =", n_valid_reflect)

In [ ]:
sign_classes_train_reflect, class_indices_train_reflect, class_counts_train_reflect = np.unique(y_train_reflect, return_index = True, return_counts = True)
sign_classes_valid_reflect, class_indices_valid_reflect, class_counts_valid_reflect = np.unique(y_valid, return_index = True, return_counts = True)
sign_classes_test_reflect, class_indices_test_reflect, class_counts_test_reflect = np.unique(y_test, return_index = True, return_counts = True)

print("SIGN OCCURRENCES IN DATASETS WITH REFLECTIONS")
N = n_classes
ind = np.arange(n_classes)  # the x locations for the groups
width = 0.2       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, class_counts_train_reflect, width, color='r')
rects2 = ax.bar(ind + width, class_counts_valid_reflect, width, color='g')
rects3 = ax.bar(ind + 2 * width, class_counts_test_reflect, width, color='b')

# add some text for labels, title and axes ticks
ax.set_ylabel('Occurrences')
ax.set_title('SIGN OCCURRENCES IN DATASETS WITH REFLECTIONS')
ax.set_xticks(ind + width)
x_tick_labels = []
for index in range(n_classes):
    if index % 5 == 0:
        x_tick_labels.append(str(index))
    else:
        x_tick_labels.append('')
ax.set_xticklabels(x_tick_labels)

ax.legend((rects1[0], rects2[0], rects3[0]), ('train', 'valid', 'test'))


def autolabel(rects):
    """
    Attach a text label above each bar displaying its height
    """
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

#autolabel(rects1)
#autolabel(rects2)
#autolabel(rects3)
ax.axis("on")

plt.show()

In [ ]:
# rotate images
from skimage.transform import rotate
from skimage.transform import warp
from skimage.transform import ProjectiveTransform

def rotate(X, intensity):
    for index in range(X.shape[0]):
        delta = 30.0 * intensiry # scale with additive intensity
        X[index] = rotate(X[index], random.uniform(-delta, delta), mode = 'edge')
    return X

def apply_projection_transformation(X, intensity):
    image_size = X.shape[1]
    d = image_size * 0.3 * intensity
    for index in range(image_size_z):
        top_left_corner_top_margin = random.uniform(-d, d)
        top_left_corner_left_margin = random.uniform(-d, d)
        top_right_corner_top_margin = random.uniform(-d, d)
        top_right_corner_right_margin = random.uniform(-d, d)
        bottom_left_corner_bottom_margin = random.uniform(-d, d)
        bottom_left_corner_left_margin = random.uniform(-d, d)
        bottom_right_corner_bottom_margin = random.uniform(-d, d)
        bottom_right_corner_right_margin = random.uniform(-d, d)
        
        transform = ProjectiveTransform()
        transform.estimate(np.array((
                    (top_left_corner_left_margin, top_left_corner_top_margin),
                    (image_size - bottom_left_corner_left_margin, image_size - bottom_left_corner_bottom_margin),
                    (image_size - bottom_right_corner_right_margin, image_size - bottom_right_corner_bottom_margin)
                    (top_right_corner_right_margin, top_right_corner_top_margin),
                )), np.array((
                    (0, 0),
                    (0, image_size),
                    (image_size, image_size),
                    (image_size, 0),
                )))
        X[index] = warp(X[index], trasofrm, output_shape=(image_size, image_size), order = 1, mode = 'edge')
        
    return X


In [ ]:
from sklearn.utils import shuffle

X_train_reflect, y_train_reflect = shuffle(X_train_reflect, y_train_reflect)

In [ ]:
print(X_train_reflect[0].shape)

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

In [ ]:
import tensorflow as tf

EPOCHS = 74
BATCH_SIZE = 128

In [ ]:
from tensorflow.contrib.layers import flatten

# X.shape = (32, 32, 1)

def LeNet(x):    
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 32x32x32.
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(32))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='SAME') + conv1_b

    # Activation.
    conv1 = tf.nn.relu(conv1)

    # Pooling. Input = 32x32x32. Output = 16x16x32.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Layer 2: Convolutional. Input = 16x16x32. Output = 16x16x64.
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 32, 64), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(64))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='SAME') + conv2_b
    
    # Activation.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 16x16x64. Output = 8x8x64.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Layer 3: Convolutional. Input = 8x8x64. Output = 8x8x128.
    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 64, 128), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(128))
    conv3   = tf.nn.conv2d(conv2, conv3_W, strides=[1, 1, 1, 1], padding='SAME') + conv3_b
    
     # Activation.
    conv3 = tf.nn.relu(conv3)

    # Pooling. Input = 8x8x128. Output = 4x4x128.
    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # Flatten. Input = 4x4x128. Output = 2048.
    fc0   = flatten(conv3)
    
    # Layer 4: Fully Connected. Input = 2048. Output = 1024.
    fc1_W = tf.Variable(tf.truncated_normal(shape=(2048, 1024), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(1024))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
    
    # Activation.
    logits    = tf.nn.relu(fc1)

    # Layer 5: Fully Connected. Input = 1024. Output = 43.
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(1024, n_classes), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(n_classes))
    logits = tf.matmul(fc1, fc2_W) + fc2_b
    
    return logits

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

In [ ]:
x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

In [ ]:
rate = 0.001

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train_reflect)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train_reflect, y_train_reflect = shuffle(X_train_reflect, y_train_reflect)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train_reflect[offset:end], y_train_reflect[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_valid_reflect, y_valid_reflect)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

In [14]:
import numpy as np
import cv2

my_image_file_name_prefix = 'my-traffic-signs-data/image_'
my_image_file_name_postfix = '.png'

n_images = 9

for index in range(n_images):
    my_f_0 = cv2.imread(my_image_file_name_prefix + str(index) + my_image_file_name_postfix, cv2.COLOR_RGBA2BGR)
    print(my_f_0)
#cv2.imshow('image', my_f_0)



[[[195 140  84]
  [195 140  84]
  [194 139  84]
  ..., 
  [199 146  90]
  [199 146  90]
  [199 147  90]]

 [[196 142  86]
  [195 141  86]
  [196 142  87]
  ..., 
  [199 147  92]
  [199 147  92]
  [199 147  92]]

 [[198 144  88]
  [197 143  88]
  [197 143  88]
  ..., 
  [200 148  94]
  [201 149  94]
  [201 149  94]]

 ..., 
 [[223 181 143]
  [222 180 142]
  [222 180 143]
  ..., 
  [224 183 147]
  [224 184 146]
  [224 183 146]]

 [[223 182 146]
  [223 181 144]
  [223 182 145]
  ..., 
  [225 186 149]
  [225 187 148]
  [225 187 149]]

 [[224 185 148]
  [223 184 146]
  [223 184 146]
  ..., 
  [225 188 150]
  [226 188 150]
  [226 188 151]]]
[[[ 84  82  77]
  [ 59  58  54]
  [ 65  64  61]
  ..., 
  [ 96  97  97]
  [ 60  66  65]
  [ 87  90  88]]

 [[ 85  81  78]
  [ 60  57  55]
  [ 61  64  61]
  ..., 
  [ 91  93  94]
  [ 57  60  60]
  [ 95  97  95]]

 [[ 80  80  72]
  [ 58  58  54]
  [ 57  62  58]
  ..., 
  [105 106 105]
  [ 93  92  93]
  [115 115 114]]

 ..., 
 [[ 70  76  76]
  [ 65  64  62]


In [8]:
import cv2

my_image_file_name_prefix = 'my-traffic-signs-data/image_'
my_image_file_name_postfix = '.png'

n_images = 9
X_my_signs = np.empty([0, image_size, image_size, 3], dtype = np.int8)
for index in range(n_images):
    file_name = my_image_file_name_prefix + str(index) + my_image_file_name_postfix
    image = cv2.imread(file_name, cv2.COLOR_RGBA2BGR)
    X_my_signs.append(image)

y_my_signs = [
    40, # 0 rounadabout
    20, # 1 double curve
    39, # 2 keep left
    17, # 3 no entry
    24, # 4 road narrows on right
    34, # 5 turn left ahead
    28, # 6 children crossing
    0, # 7 speed limit (20km/h)
    13, # 8 yield  

SyntaxError: unexpected EOF while parsing (<ipython-input-8-3b87a1b2b243>, line 22)

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tk.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

---

## Step 4: Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it maybe having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")

### Question 9

Discuss how you used the visual output of your trained network's feature maps to show that it had learned to look for interesting characteristics in traffic sign images


**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 